# Łączenie danych lokalizacyjnych osób lajkujących osoby z odpowiednim nastawieniem do szczepień

In [1]:
import pandas as pd

In [2]:
df_tweet = pd.read_json('data/data_with_liking_users_labeled.json')
dict_location = pd.read_pickle('data/locations_geocoded.pkl')

## Wyciąganie informacji od Liking_users

In [3]:
df_location_liking_users= df_tweet[['liking_user_id', 
                                    'location', 
                                    'SentimentPL_clear_full_text_without_hashtags',
                                    'SentimentPL_clear_full_text_with_hashtags',
                                    'SentimentPL_raw_full_text','AutoML_label_for_3_classes_ExtraTrees']]

dict3classes_EN = {-1:'Przeciwnicy', 0: 'NEITHER', 1:'Zwolennicy'}
df_location_liking_users['y3classes_numeric'] = df_location_liking_users['AutoML_label_for_3_classes_ExtraTrees']
df_location_liking_users['y3classes_str'] = df_location_liking_users['AutoML_label_for_3_classes_ExtraTrees']
df_location_liking_users= df_location_liking_users.drop(columns=['AutoML_label_for_3_classes_ExtraTrees'])
df_location_liking_users= df_location_liking_users.replace({'y3classes_str': dict3classes_EN})
df_location_liking_users= df_location_liking_users[df_location_liking_users['y3classes_numeric'] != 0]
df_location_liking_users= df_location_liking_users[df_location_liking_users.location != ""].dropna(axis=0)

list_of_coordinate = []
list_of_location = []
i = 0
for row in df_location_liking_users.iterrows():
    try:
        info_location = dict_location[row[1]['location']]
        new_coordinate = info_location[-1]
        if type(new_coordinate) == tuple:
            i += 0
            list_of_location.append(info_location)
            list_of_coordinate.append(new_coordinate)
        else:
            list_of_coordinate.append("")
            list_of_location.append("")
            print(info_location)
    except:
        list_of_coordinate.append("")
        list_of_location.append("")

print(f'Tabela ma rozmiar {df_location_liking_users.shape}, a otrzymany zbiór coordynatów: {len(list_of_coordinate)}')

df_location_liking_users['location'] = list_of_location
df_location_liking_users['coordinate'] = list_of_coordinate
df_location_liking_users= df_location_liking_users[df_location_liking_users['coordinate'] != ""]
df_location_liking_users['latitude'] = df_location_liking_users['coordinate'].apply(lambda x: x[0])
df_location_liking_users['longitude'] = df_location_liking_users['coordinate'].apply(lambda x: x[1])
df_location_liking_users['country'] = df_location_liking_users['location'].apply(lambda x: x[-2].split(', ')[-1])

df_stance_liking_users = df_location_liking_users.groupby(by=['liking_user_id'])['y3classes_numeric'].sum().reset_index()
df_stance_liking_users.columns = ['liking_user_id', 'y3classes_sum']
df_stance_liking_users['y3classes_str_general'] = ['Zwolennicy' if sum_stance >= 0 else "Przeciwnicy" for sum_stance in df_stance_liking_users['y3classes_sum']]
df_stance_liking_users = df_stance_liking_users[df_stance_liking_users['y3classes_sum'] != 0]

df_loc_liking_users = pd.merge(left=df_location_liking_users, right=df_stance_liking_users, on='liking_user_id').drop_duplicates(subset=['liking_user_id']).reset_index(drop=True)

C:\Users\MaciekM\AppData\Local\Temp/ipykernel_7216/4105066461.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location_liking_users['y3classes_numeric'] = df_location_liking_users['AutoML_label_for_3_classes_ExtraTrees']
C:\Users\MaciekM\AppData\Local\Temp/ipykernel_7216/4105066461.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location_liking_users['y3classes_str'] = df_location_liking_users['AutoML_label_for_3_classes_ExtraTrees']


Tabela ma rozmiar (128332, 7), a otrzymany zbiór coordynatów: 128332


In [5]:
df_loc_liking_users.columns= ['id', 'location', 'SentimentPL_clear_full_text_without_hashtags',
       'SentimentPL_clear_full_text_with_hashtags',
       'SentimentPL_raw_full_text', 'y3classes_numeric', 'y3classes_str',
       'coordinate', 'latitude', 'longitude', 'country', 'y3classes_sum',
       'y3classes_str_general']

## Wyciąganie informacji dla użytkowników tweetujących

In [6]:
df_tweet.columns

Index(['tweet_id', 'liking_user_id', 'location', 'created_at', 'id', 'id_str',
       'full_text', 'truncated', 'main_hashtags', 'user_mention',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user_id', 'user_id_str', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
       'conversation_id', 'favorited', 'retweeted', 'possibly_sensitive',
       'lang', 'bert_proba_0', 'bert_proba_1', 'bert_label',
       'xg_boost_proba_0', 'xg_boost_proba_1', 'xg_boost_labels', 'cluster',
       'sentiment_tag', 'stance_tag',
       'SentimentPL_clear_full_text_without_hashtags',
       'SentimentPL_clear_full_text_with_hashtags',
       'SentimentPL_raw_full_text', 'AutoML_label_for_3_classes_ExtraTrees',
       'AutoML_label_for_2_classes_Ensemble'],
      dtype='object')

In [7]:
df_location_tweeting_users= df_tweet[['user_id', 
                                    'location', 
                                    'SentimentPL_clear_full_text_without_hashtags',
                                    'SentimentPL_clear_full_text_with_hashtags',
                                    'SentimentPL_raw_full_text','AutoML_label_for_3_classes_ExtraTrees']]

dict3classes_EN = {-1:'Przeciwnicy', 0: 'NEITHER', 1:'Zwolennicy'}
df_location_tweeting_users['y3classes_numeric'] = df_location_tweeting_users['AutoML_label_for_3_classes_ExtraTrees']
df_location_tweeting_users['y3classes_str'] = df_location_tweeting_users['AutoML_label_for_3_classes_ExtraTrees']
df_location_tweeting_users= df_location_tweeting_users.drop(columns=['AutoML_label_for_3_classes_ExtraTrees'])
df_location_tweeting_users= df_location_tweeting_users.replace({'y3classes_str': dict3classes_EN})
df_location_tweeting_users= df_location_tweeting_users[df_location_tweeting_users['y3classes_numeric'] != 0]
df_location_tweeting_users= df_location_tweeting_users[df_location_tweeting_users.location != ""].dropna(axis=0)

list_of_coordinate = []
list_of_location = []
i = 0
for row in df_location_tweeting_users.iterrows():
    try:
        info_location = dict_location[row[1]['location']]
        new_coordinate = info_location[-1]
        if type(new_coordinate) == tuple:
            i += 0
            list_of_location.append(info_location)
            list_of_coordinate.append(new_coordinate)
        else:
            list_of_coordinate.append("")
            list_of_location.append("")
            print(info_location)
    except:
        list_of_coordinate.append("")
        list_of_location.append("")

print(f'Tabela ma rozmiar {df_location_tweeting_users.shape}, a otrzymany zbiór coordynatów: {len(list_of_coordinate)}')

df_location_tweeting_users['location'] = list_of_location
df_location_tweeting_users['coordinate'] = list_of_coordinate
df_location_tweeting_users= df_location_tweeting_users[df_location_tweeting_users['coordinate'] != ""]
df_location_tweeting_users['latitude'] = df_location_tweeting_users['coordinate'].apply(lambda x: x[0])
df_location_tweeting_users['longitude'] = df_location_tweeting_users['coordinate'].apply(lambda x: x[1])
df_location_tweeting_users['country'] = df_location_tweeting_users['location'].apply(lambda x: x[-2].split(', ')[-1])

df_stance_tweeting_users = df_location_tweeting_users.groupby(by=['user_id'])['y3classes_numeric'].sum().reset_index()
df_stance_tweeting_users.columns = ['user_id', 'y3classes_sum']
df_stance_tweeting_users['y3classes_str_general'] = ['Zwolennicy' if sum_stance >= 0 else "Przeciwnicy" for sum_stance in df_stance_tweeting_users['y3classes_sum']]
df_stance_tweeting_users = df_stance_tweeting_users[df_stance_tweeting_users['y3classes_sum'] != 0]

df_loc_tweeting_users = pd.merge(left=df_location_tweeting_users, right=df_stance_tweeting_users, on='user_id').drop_duplicates(subset=['user_id']).reset_index(drop=True)

C:\Users\MaciekM\AppData\Local\Temp/ipykernel_7216/3609452535.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location_tweeting_users['y3classes_numeric'] = df_location_tweeting_users['AutoML_label_for_3_classes_ExtraTrees']
C:\Users\MaciekM\AppData\Local\Temp/ipykernel_7216/3609452535.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location_tweeting_users['y3classes_str'] = df_location_tweeting_users['AutoML_label_for_3_classes_ExtraTrees']


Tabela ma rozmiar (128332, 7), a otrzymany zbiór coordynatów: 128332


In [8]:
df_loc_tweeting_users.columns =  ['id', 'location', 'SentimentPL_clear_full_text_without_hashtags',
       'SentimentPL_clear_full_text_with_hashtags',
       'SentimentPL_raw_full_text', 'y3classes_numeric', 'y3classes_str',
       'coordinate', 'latitude', 'longitude', 'country', 'y3classes_sum',
       'y3classes_str_general']

## Łączenie wszystkich danych

In [9]:
data = pd.concat([df_loc_liking_users, df_loc_tweeting_users], axis=0).drop_duplicates(subset=['id']).reset_index(drop=True)

In [10]:
data.shape

(22750, 13)

In [11]:
data.y3classes_str_general.value_counts()

Zwolennicy     18050
Przeciwnicy     4700
Name: y3classes_str_general, dtype: int64

In [15]:
data.country.value_counts()[:20]

Polska                         19333
United States                    761
United Kingdom                   379
臺灣                               333
Deutschland                      281
France                           173
Italia                            99
Canada                            95
Nederland                         77
日本                                73
België / Belgique / Belgien       71
España                            66
Česko                             64
أوروبا                            58
Brasil                            49
Sverige                           44
ປະເທດລາວ                          41
Australia                         38
India                             37
Россия                            30
Name: country, dtype: int64

In [16]:
data.to_json('data/location_data_all.json')